In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import cavsim
from cavsim import Measure
from cavsim.connectors import BaseConnector, Connector
from cavsim.channels import ImportChannel, ExportChannel
from cavsim.components import BaseComponent, Component
from cavsim.solvers import BaseSolver, SimpleSolver
from cavsim.fluids import Fluid

In [ ]:
from cavsim.pipes.pipe import Pipe
from cavsim.boundaries.left_boundary_pressure import LeftBoundaryPressure
from cavsim.boundaries.left_boundary_velocity import LeftBoundaryVelocity
from cavsim.boundaries.right_boundary_pressure import RightBoundaryPressure
from cavsim.boundaries.right_boundary_velocity import RightBoundaryVelocity
from cavsim.boundaries.simple_pipe_connector import PipeConnectorSimple
from cavsim.boundaries.zeta_joint import ZetaJoint
from cavsim.boundaries.simple_t_joint import SimpleTJoint
from cavsim.boundaries.simple_right_gasdampener import RightGasBubbleSimple
from cavsim.boundaries.frictionless_gasdampener import FrictionlessGasDampener
from cavsim.boundaries.gasdampener import GasDampener
from cavsim.boundaries.pump_suction_valve import PumpSuctionValve

In [ ]:
fluid = Fluid(1000, 1e-3, 2.08e9, 2.3e-3,initial_pressure=328000)

In [ ]:
def pump(time):
    return np.maximum(0.16 * np.sin(70/60*(1 * time * 2.0 * np.pi)), 0.0)
    #return 0.001

In [ ]:
def step(time):
    return 0.35 * (time > 0.25)

In [ ]:
pipe1 = Pipe(0.056, 7.0, 0.002, 2e11, 1e-7, 500)
pipe2 = Pipe(0.056, 7.0, 0.002, 2e11, 1e-7, 500)

In [ ]:
lb1 = LeftBoundaryPressure(328000)
pipe1.connect(lb1)

In [ ]:
rb = RightBoundaryVelocity(pump)

In [ ]:
pipe2.connect(rb)

In [ ]:
angle = np.pi / 2.0

In [ ]:
valve = PumpSuctionValve(7950.0, 7.0, 6570.0, 0.022, 0.068, 0.041, 0.037, angle, 0.0076, 6.480, 107.00, 74.00, 1.40, -2.40, 0.50, 0.0, 0.0, 30.0e-3)
valve.connect(pipe1)
valve.connect(pipe2)

In [ ]:
#valve = ZetaJoint(1.0)
#pipe1.connect(valve)
#pipe2.connect(valve)

In [ ]:
plogs1 = []
plogs2 = []
vlogs = []
vlogs1 = []
flogs = []
rblogs = []
relogs = []
dislog = []
springlog = []
contact_forcelog = []
damping_force = []
upper_pforce = []
lower_pforce = []
gforce = []
flow_force = []
velocity = []


def logging():
    dislog.append(valve.field_wide_slice('displacement', 1) + 0.0)
    velocity.append(valve.field_wide_slice('valve_velocity', 1) + 0.0)
    gforce.append(valve.field_wide_slice('gravity_force', 1) + 0.0)
    springlog.append(valve.field_wide_slice('spring_force', 1) + 0.0)
    upper_pforce.append(valve.field_wide_slice('upper_pressure_force', 1) + 0.0)
    lower_pforce.append(valve.field_wide_slice('lower_pressure_force', 1) + 0.0)
    damping_force.append(valve.field_wide_slice('damping_force', 1) + 0.0)
    flow_force.append(valve.field_wide_slice('damping_force', 1) + 0.0)
    contact_forcelog.append(valve.field_wide_slice('contact_pressure_force', 1) + 0.0)
    plogs1.append(pipe1.field_wide_slice('pressure', 0) + 0.0)
    plogs2.append(pipe2.field_wide_slice('pressure', 0) + 0.0)
    vlogs.append(pipe2.field_wide_slice('velocity', 0) + 0.0)
    flogs.append(pipe2.field_wide_slice('friction_steady', 0) + 0.0)
    rblogs.append(rb._velocity[0,1])
    relogs.append(pipe2.field_wide_slice('reynolds', 0) + 0.0)

In [ ]:
solver = SimpleSolver()
solver.fluid = fluid
solver.seeds = [pipe1, pipe2]
solver._callback = logging

In [ ]:
solver.solve(1e20, 0.157, 3)

In [ ]:
pfield1 = np.stack(plogs1)
pfield2 = np.stack(plogs2)
vfield = np.stack(vlogs)
#ffield = np.stack(flogs)
rbfield = np.stack(rblogs)
refield = np.stack(relogs)
disfield = np.stack(dislog)
dampingfield = np.stack(damping_force)
lowp_field = np.stack(lower_pforce)
upperp_field = np.stack(upper_pforce)
contact_pressure = np.stack(contact_forcelog)
springforce = np.stack(springlog)
gfield = np.stack(gforce)
pfield2.shape
flow_field = np.stack(flow_force)
velfield = np.stack(velocity)

In [ ]:
time = np.linspace(0, 1.0, int(pfield1[:,-2].shape[0]))
plt.figure(figsize=(16, 10))
#plt.subplot(1, 2, 1)
#plt.plot(vfield[:,0])
#plt.plot(1, 2, 2)
val1 = 1500
val = -1
#upper_force = upperp_field[val1:val, 0] + gfield[val1:val, 0] + springforce[val1:val, 0]
#lower_force = lowp_field[val1:val, 0] + contact_pressure[val1:val, 0]
#plt.subplot(1, 2, 2)
#plt.plot(time[val1:val], upperp_field[val1:val, 0], color='red')
#plt.plot(time[val1:val], lowp_field[val1:val, 0], color='green')
#plt.plot(time[val1:val], dampingfield[val1:val, 0], color='blue')
#plt.plot(time[val1:val], flow_field[val1:val, 0], color='black')
#plt.plot(time[val1:val], contact_pressure[val1:val, 0], color='black')
#plt.plot(time[val1:val], springforce[val1:val, 0], color='grey')
#plt.plot(time[val1:val], upper_force, color='red')
#plt.plot(time[val1:val], lower_force, color='blue')
#for i in range(upper_force.shape[0]):
#    if upper_force[i] > lower_force[i]:
#        print("True")
#    else:
#        print("False")

#plt.plot(time[val1:val], pfield2[val1:val, 1], color='red')
#plt.plot(time[val1:val], pfield1[val1:val, -2])

#plt.xlim(time[val1],time[val])
#plt.ylim(-1e5, 5e5)

#plt.subplot(1, 2, 1)
plt.plot(time[val1:val], disfield[val1:val, 0], color='black')
plt.plot(time[val1:val], velfield[val1:val, 0], color='red')
#plt.plot(time[val1:val], pfield2[val1:val, 1], color='red')
#plt.plot(time[val1:val], pfield1[val1:val, -2], color='green')
#plt.ylim(0, 40.0e5)
plt.show()

In [ ]:
#plt.plot(time[val1:val], pump(time[val1:val]))
#print(np.max(disfield[val1:val, 0]*1000))

In [ ]:
valve.field('spring_force')[:3, 0]

In [ ]:
valve.field('displacement')[:, 0]

In [ ]:
valve.field('valve_velocity')[:3, 0]

In [ ]:
valve.field('acceleration')[:3, 0]

In [ ]:
valve.field('gravity_force')[:3, 0]

In [ ]:
valve.field('acceleration_force')[:3, 0]

In [ ]:
valve.field('damping_force')[:3, 0]

In [ ]:
valve.field('upper_pressure_force')[:3, 0]

In [ ]:
valve.field('lower_pressure_force')[:3, 0]

In [ ]:
valve.field('contact_pressure_force')[:3, 0]

In [ ]:
valve.field('flow_force')[:3, 0]

In [ ]:
valve.field('valve_zeta')[:3, 0]

In [ ]:
valve._flow_constant_1

In [ ]:
valve._flow_constant_2

In [ ]:
valve._valve_zeta

In [ ]:
valve.field('pressure')[:3, 0]

In [ ]:
pipe2.field('pressure')[:3, 1]

In [ ]:
valve._cases[160:170]

In [ ]:
valve._lower_pressure_force[:,0]

In [ ]:
valve.field('lower_pressure_force')

In [ ]:
valve.field_wide_slice('lower_pressure_force', 1)

In [ ]:
lowp_field.shape

In [ ]:
valve.calculate_lower_pressure_force()